In [5]:
import pandas as pd
import pickle 
import sys
sys.path.append("/Ray_finrl/findrl_ray/finenv")
sys.path.append("./FinRL")
import finrl
from finenv.env_stocktrading import StockTradingEnv
from finenv.preprocessors import data_split
from finenv.save_model import upload_files
import psutil
import ray
from datetime import datetime
ray._private.utils.get_system_memory = lambda: psutil.virtual_memory().total
from ray.tune.registry import register_env
from gymnasium.wrappers import EnvCompatibility
# load the DataFrame from a pickle file
#df = pd.read_pickle('dataset/nasdaq74_proc_test.pkl')
train = pd.read_csv('dataset/train_data.csv')
train = train.set_index(train.columns[0])
train.index.names = ['']

#Convert date from pickle formats. 
#train = pd.read_pickle('dataset/qqq_proc.pkl')
#train['date'] = pd.to_datetime(train['date'])
#train['date'] = train['date'].dt.strftime('%Y-%m-%d %H:%M')

INDICATORS = ['macd','boll_ub','boll_lb','rsi_30','cci_30','dx_30','close_30_sma','close_60_sma']
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 82, State Space: 821


In [2]:
#INDICATORS = ['macd','rsi','cci','adx']

In [5]:
train.head()
train.dtypes

date             object
tic              object
open            float64
high            float64
low             float64
close           float64
volume          float64
day             float64
macd            float64
boll_ub         float64
boll_lb         float64
rsi_30          float64
cci_30          float64
dx_30           float64
close_30_sma    float64
close_60_sma    float64
vix             float64
turbulence      float64
dtype: object

In [6]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

In [7]:
def env_creator(env_config):
    # env_config is passed as {} and defaults are set here
    df = env_config.get('df', train)
    hmax = env_config.get('hmax', 200)
    initial_amount = env_config.get('initial_amount', 1000000)
    num_stock_shares = env_config.get('num_stock_shares', [0] * stock_dimension)
    buy_cost_pct = env_config.get('buy_cost_pct', buy_cost_list)
    sell_cost_pct = env_config.get('sell_cost_pct', sell_cost_list)
    state_space = env_config.get('state_space', 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension)
    stock_dim = env_config.get('stock_dim', stock_dimension)
    tech_indicator_list = env_config.get('tech_indicator_list', INDICATORS)
    action_space = env_config.get('action_space', stock_dimension)
    reward_scaling = env_config.get('reward_scaling', 1e-2)

    return EnvCompatibility(StockTradingEnv(
        df=df,
        hmax=hmax,
        initial_amount=initial_amount,
        num_stock_shares=num_stock_shares,
        buy_cost_pct=buy_cost_pct,
        sell_cost_pct=sell_cost_pct,
        state_space=state_space,
        stock_dim=stock_dim,
        tech_indicator_list=tech_indicator_list,
        action_space=action_space,
        reward_scaling=reward_scaling
    ))

register_env("finrl", env_creator)

In [8]:
from ray.rllib.agents import ppo
ray.shutdown()
#ray.init(num_cpus=45,num_gpus=4,dashboard_port=8300)
print(f"ray is being initialized")

ray is being initialized


In [9]:
config = ppo.PPOConfig()
config = config.environment(env_config={'hmax':500,'initial_amount':1000000})
config = config.training(gamma=0.9, lr=0.001, kl_coeff=0.3)  
config = config.resources(num_gpus=0)
config = config.rollouts(num_rollout_workers=12) 
config = config.framework(framework="torch")
config['seed'] = 42
config["model"]["fcnet_hiddens"] = [256, 256, 128,16]
config['sgd_minibatch_size'] = 256
config['num_sgd_iter'] = 30
config['train_batch_size'] = 10000

In [10]:
# registering the environment to ray
#trainer = config.build(env="finrl") 
trainer = ppo.PPOTrainer(env='finrl', config=config)

2023-04-05 10:37:46,011	WARNING deprecation.py:50 -- DeprecationWarning: `algo = Algorithm(env='finrl', ...)` has been deprecated. Use `algo = AlgorithmConfig().environment('finrl').build()` instead. This will raise an error in the future!
2023-04-05 10:37:53,944	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


KeyboardInterrupt: 

In [ ]:
from tqdm.notebook import tqdm
# Train away -------------------------------------------------------------
total_episodes = 25 
agent_name = 'ppo'
ep = 0
results = []
bar = tqdm(total=total_episodes, desc="Episodes")
date = datetime.now().strftime('%y%m%d')

while ep <= total_episodes:
    results.append(trainer.train())
    ep += 1
    bar.update(n=1)
    rwd = results[-1]['episode_reward_mean']
    if ep % 5 == 0:
        print(f'Mean Rwd:{rwd}')
    if ep % 25 == 0:
        #cwd_checkpoint = "results/checkpoints/" +  + '_' + str(ep)
        cwd_checkpoint = f"model/{agent_name}_{date}"
        trainer.save(cwd_checkpoint)
        print(f"Checkpoint{ep} saved in directory {cwd_checkpoint}")
bar.close()

2023-04-05 10:46:07,781	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
(RolloutWorker pid=31975) 2023-04-05 10:46:16,799	ERROR worker.py:772 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=31975, ip=172.28.231.77, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f70d614fcd0>)
(RolloutWorker pid=31975)   File "/home/ga_aiot/anaconda3/envs/finrl/lib/python3.8/site-packages/gymnasium/envs/registration.py", line 569, in make
(RolloutWorker pid=31975)     _check_version_exists(ns, name, version)
(RolloutWorker pid=31975)   File "/home/ga_aiot/anaconda3/envs/finrl/lib/python3.8/site-packages/gymnasium/envs/registration.py", line 219, in _check_version_exists
(RolloutWorker pid=31975)     _check_name_exists(ns, name)
(RolloutWorker pid=31975)   File "/home/ga_aiot/anaconda3/envs/finrl/lib/python3.8/site-packages/gymnasium/envs/registra

EnvError: The env string you provided ('finrl') is:
a) Not a supported/installed environment.
b) Not a tune-registered environment creator.
c) Not a valid env class string.

Try one of the following:
a) For Atari support: `pip install gym[atari] autorom[accept-rom-license]`.
   For VizDoom support: Install VizDoom
   (https://github.com/mwydmuch/ViZDoom/blob/master/doc/Building.md) and
   `pip install vizdoomgym`.
   For PyBullet support: `pip install pybullet`.
b) To register your custom env, do `from ray import tune;
   tune.register('[name]', lambda cfg: [return env obj from here using cfg])`.
   Then in your config, do `config['env'] = [name]`.
c) Make sure you provide a fully qualified classpath, e.g.:
   `ray.rllib.examples.env.repeat_after_me_env.RepeatAfterMeEnv`


In [ ]:
# save the trained agent
#cwd_checkpoint = "model/" + str(agent_name) + '_' + str(ep)
cwd_checkpoint = "model/"
trainer.save(cwd_checkpoint)